In [36]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import tensorflow as tf
import MIDASpy as md


In [37]:
column_names=['Column 1','Column 2','Column 3','Column 4','Column 5','Column 6','Column 7','Column 8','Column 9','Column 10','Column 11','Column 12','Column 13','Column 14','Column 15','Column 16']
df=pd.read_excel(r'C:\Users\furqz\OneDrive\Desktop\Data mining stiff\incomplete\Incomplete Datasets Without Labels\HOV\HOV_AG_1.xlsx',header=None,names=column_names)
df

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\furqz\\OneDrive\\Desktop\\Data mining stiff\\incomplete\\Incomplete Datasets Without Labels\\HOV\\HOV_AG_1.xlsx'

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [ ]:
import tensorflow as tf
tf.compat.v1.reset_default_graph()

In [ ]:
import tensorflow as tf
tf.compat.v1.reset_default_graph()

In [ ]:
import tensorflow.compat.v1 as tf
tf.reset_default_graph()

In [ ]:
categorical = ['Column 1','Column 2','Column 3','Column 4','Column 5','Column 6','Column 7','Column 8','Column 9','Column 10','Column 11','Column 12','Column 13','Column 14','Column 15','Column 16']
data_cat, cat_cols_list = md.cat_conv(df[categorical])

df.drop(categorical, axis = 1, inplace = True)
constructor_list = [df]
constructor_list.append(data_cat)
data_in = pd.concat(constructor_list, axis=1)

na_loc = data_in.isnull()
data_in[na_loc] = np.nan

In [ ]:
print(data_in.head())

In [ ]:
imputer = md.Midas(layer_structure = [16,16+15,16+15+15,16+15+15+15,16+15+15,16+15,16], vae_layer = False, seed = 100, input_drop = 0.5)
imputer.build_model(data_in,softmax_columns = cat_cols_list)
imputer.train_model(training_epochs = 100)


In [ ]:
imputations = imputer.generate_samples(m=10).output_list 

In [ ]:
np.shape(imputations[-1])

In [ ]:
new_df=imputations[-1]

In [ ]:
complete_df=pd.read_excel(r'C:\Users\furqz\OneDrive\Desktop\Data mining stiff\complete\Original Datasets Without Labels\HOV.xlsx',header=None,names=column_names)
complete_df

In [ ]:
flat_cats = [cat for variable in cat_cols_list for cat in variable]

for i in range(len(imputations)):
    tmp_cat = [imputations[i][x].idxmax(axis=1) for x in cat_cols_list]
    cat_df = pd.DataFrame({categorical[i]:tmp_cat[i] for i in range(len(categorical))})
    imputations[i] = pd.concat([imputations[i], cat_df], axis = 1).drop(flat_cats, axis = 1)

In [ ]:
print(imputations[-1].columns)

In [ ]:
new_df=imputations[-1]
new_df

In [ ]:
new_df.columns

In [ ]:
def replace_column_values(dataframe):
    
    for col_name in dataframe.columns:
        data_list=dataframe[col_name]
        for index, data in enumerate(data_list):
            data=data.replace(col_name+'_','')
            data_list[index]=data
        dataframe[col_name]=data_list    
    return(dataframe)            

In [ ]:
mod_df=replace_column_values(new_df)
mod_df

In [ ]:
mod_df.dtypes

calculating AE

In [ ]:
copy_est_df=mod_df.astype('string')

In [ ]:
m=copy_est_df.astype('string')
m

In [ ]:
# I function
def I_function(copy_est_df,original_df):
    for col_name in copy_est_df.columns:
        #copy_est_df[col_name] = copy_est_df[col_name].astype('string')
        #original_df[col_name] = original_df[col_name].astype('string')
        est_data_list=copy_est_df[col_name].astype('string')
        #print(est_data_list.dtypes)
        org_data_list=original_df[col_name].astype('string')
        for index, data in enumerate(est_data_list):
            if est_data_list[index]==org_data_list[index]:
                est_data_list[index]='0'
            else:
                est_data_list[index]='1'
            
        copy_est_df[col_name]=est_data_list
    return(copy_est_df.astype('int64'))  

In [ ]:
def Sum_new(sum):
    summ=0
    for n in sum:
        summ=summ+n
    return (summ)

𝐴𝐸 = (1 /𝑛) ∑ 𝐼(𝑥̂𝑖 = 𝑥𝑖)

In [ ]:
def AE(copy_est_df,original_df):
    copy_est_df=I_function(copy_est_df,original_df)
    total_sum=copy_est_df.astype('int64').sum()
    final_sum=Sum_new(total_sum)
    ae=final_sum/(232*16)
    return(ae)

In [ ]:
AE(copy_est_df,complete_df)

In [ ]:
mod_df.to_excel("HOV_C_20(imp).xlsx", index=False,header=False)